# Libraries

In [1]:
import gc
import pandas as pd

# Data

In [2]:
dtypes = {
    'ip'            : 'uint32',
    'app'           : 'uint16',
    'device'        : 'uint16',
    'os'            : 'uint16',
    'channel'       : 'uint16',
    'is_attributed' : 'uint8',
    'click_id'      : 'uint32'
}

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'is_attributed']

In [3]:
gc.collect()

train = pd.read_csv(
    '../input/train.csv',
    nrows=130000000,
    dtype=dtypes,
    engine='c',
    low_memory=True,
    usecols=train_cols
)

gc.collect()

0

# Exploration

In [4]:
train.shape

(130000000, 6)

In [5]:
train.head()

,ip,app,device,os,channel,is_attributed
0,83230,3,1,13,379,0
1,17357,3,1,19,379,0
2,35810,3,1,13,379,0
3,45745,14,1,13,478,0
4,161007,3,1,13,379,0


# Preprocessing

In [6]:
# def dataPreProcessTime(df):
#     df['click_time'] = pd.to_datetime(df['click_time']).dt.date
#     df['click_time'] = df['click_time'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
    
# train = dataPreProcessTime(train)

In [7]:
train.head()

,ip,app,device,os,channel,is_attributed
0,83230,3,1,13,379,0
1,17357,3,1,19,379,0
2,35810,3,1,13,379,0
3,45745,14,1,13,478,0
4,161007,3,1,13,379,0


# Train/Test Split

In [8]:
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
features = ['ip', 'app', 'device', 'os', 'channel']
target = ['is_attributed']

X = train[features]
y = train[target]

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.01)

# Model

In [11]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import xgboost as xgb
import lightgbm as lgbm

from sklearn.metrics import log_loss, auc

In [12]:
gc.collect()

#model = XGBClassifier(n_jobs=1, n_estimators=2, learning_rate=0.05, max_depth=2)
model = LGBMClassifier(n_jobs=-1, n_estimators=500, learning_rate=0.05, max_depth=10, subsample=0.7)

model.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MemoryError: 

In [ ]:
y_prob = model.predict_proba(X_val)

print('Log loss:', log_loss(y_val, y_prob))

# Generate Submission

In [ ]:
del train, X_train, X_val, y_train, y_val
gc.collect()

In [ ]:
test = pd.read_csv(
    '../input/test.csv',
    engine='c',
    low_memory=True,
)

test_cols = ['ip', 'app', 'device', 'os', 'channel']
X_test = test[test_cols]

In [ ]:
submission = pd.DataFrame()
submission['click_id'] = test['click_id']
submission['is_attributed'] = model.predict_proba(X_test)[:,1]

In [ ]:
submission.head()

In [ ]:
submission.shape

In [ ]:
submission.to_csv('../submissions/lgb_simple_v4.csv', index=False)